# Medical Question-Answer Dataset Curation Using Label Studio

In this tutorial, we'll explore the process of curating a dataset for a Generative AI model, specifically focusing on medical question and answer generation using Label Studio. This involves setting up projects in Label Studio, importing datasets, and configuring tasks for question and answer generation.

### Installation
To set up your environment, install the required libraries using pip:

In [ ]:
!pip install -r requirements.txt

## Setup Label Studio
Start Label Studio and connect to it with the SDK. You can retrieve an API key from the user settings. 

```
$ label-studio
```

In [15]:
# Import the SDK and the client module
from label_studio_sdk import Client

# Define the URL where Label Studio is accessible and the API key for your user account
LABEL_STUDIO_URL = 'http://localhost:8080'
API_KEY = '<your_api_key>'

# Connect to the Label Studio API and check the connection
client = Client(url=LABEL_STUDIO_URL, api_key=API_KEY)
client.check_connection()

{'status': 'UP'}

## MedChat Dataset
In this section, we will import the [MedChat QA dataset](https://huggingface.co/datasets/ngram/medchat-qa) from Hugging Face datasets and set up a Label Studio project to curate it.

In [39]:
from datasets import load_dataset

medchat_dataset = load_dataset("ngram/medchat-qa")

In [41]:
medchat_dataset['train'][1]

{'question': 'What is one of the methods of administering this drug, as mentioned in the section?',
 'answer': 'For intravenous infusion into a peripheral or central vein.'}

### Creating a Label Studio Project
Next, create a new project in Label Studio for this dataset:

In [35]:
medchat_project = client.start_project(
    title='Project 1: MedChat',
    color='#008000',
    label_config='''
<View className="root">
  <Style>
  .root {
    font-family: 'Roboto', sans-serif;
    line-height: 1.6;
    background-color: #f0f0f0;
  }
  .container {
    margin: 0 auto;
    padding: 20px;
    background-color: #ffffff;
    border-radius: 5px;
    box-shadow: 0 4px 8px 0 rgba(0, 0, 0, 0.1), 0 6px 20px 0 rgba(0, 0, 0, 0.1);
  }
  .prompt {
    padding: 20px;
    background-color: #0084ff;
    color: #ffffff;
    border-radius: 5px;
    margin-bottom: 20px;
    box-shadow: 0 2px 4px 0 rgba(0, 0, 0, 0.1), 0 3px 10px 0 rgba(0, 0, 0, 0.1);
  }
  .prompt-input {
    flex-basis: 49%;
    padding: 20px;
    background-color: rgba(44, 62, 80, 0.9);
    color: #ffffff;
    border-radius: 5px;
    box-shadow: 0 2px 4px 0 rgba(0, 0, 0, 0.1), 0 3px 10px 0 rgba(0, 0, 0, 0.1);
    width: 100%;
    border: none;
    font-family: 'Roboto', sans-serif;
    font-size: 16px;
    outline: none;
  }
  .prompt-input:focus {
    outline: none;
  }
  .prompt-input:hover {
    background-color: rgba(52, 73, 94, 0.9);
    cursor: pointer;
    transition: all 0.3s ease;
  }
  .lsf-richtext__line:hover {
    background: unset;
  }
  </Style>
  <Text name="chat" value="$question" layout="dialogue" />
  <Header value="Answer:"/>
  	<Text name="summary" value="$answer" toName="summary" rows="4" editable="false" maxSubmissions="1" showSubmitButton="false"/>
  <Header value="User prompt:" />
  <View className="prompt">
  <TextArea name="prompt" toName="chat" rows="4" editable="true" maxSubmissions="1" showSubmitButton="true" />
  </View>
  <Header value="Bot answer:"/>
    <TextArea name="response" toName="chat" rows="4" editable="false" maxSubmissions="1" showSubmitButton="false" />
</View>
    '''
)

This project setup also includes the capability to configure the Label Studio ML Backend for using an LLM to help label the data. 

To set up the ML Backend, see the [LLM Interactive Example](https://github.com/HumanSignal/label-studio-ml-backend/tree/master/label_studio_ml/examples/llm_interactive).

```bash
git clone https://github.com/HumanSignal/label-studio-ml-backend.git
cd label-studio-ml-backend/label_studio_ml/examples/llm_interactive
```

We can edit the `docker-compose.yml` to point to the model we are interested in. For instance if we want to configure it to use Llama 3 running with Ollama, we can configure the following fields: 

```yaml
OPENAI_PROVIDER=ollama
OPENAI_MODEL=llama3
OLLAMA_ENDPOINT=http://host.docker.internal:11434/v1/
```

Insert the data into the Label Studio project. 

In [ ]:
# task = medchat_dataset['train'][1]
medchat_tasks = []
for t in medchat_dataset['train']:
    medchat_tasks.append(t)
medchat_project.import_tasks(medchat_tasks) 

Notice how even this curated dataset has examples that may not be necessarily desirable. 

## Question Generation with MeDAL 
The [MeDAL dataset](https://huggingface.co/datasets/medal) is a comprehensive medical text collection curated specifically for abbreviation disambiguation, aiding in natural language understanding pre-training within the medical domain.

We can use this dataset to provide context for creating a synthetic Q&A dataset. We'll first start with a Label Studio project to generate questions. 

Set up a new project for question generation: 

In [29]:
medal_questions_project = client.start_project(
    title='Project 2: MeDAL Question Generation',
    label_config='''
<View className="root">
  <Style>
  .root {
    font-family: 'Roboto', sans-serif;
    line-height: 1.6;
    background-color: #f0f0f0;
  }
  .container {
    margin: 0 auto;
    padding: 20px;
    background-color: #ffffff;
    border-radius: 5px;
    box-shadow: 0 4px 8px 0 rgba(0, 0, 0, 0.1), 0 6px 20px 0 rgba(0, 0, 0, 0.1);
  }
  .prompt {
    padding: 20px;
    background-color: #0084ff;
    color: #ffffff;
    border-radius: 5px;
    margin-bottom: 20px;
    box-shadow: 0 2px 4px 0 rgba(0, 0, 0, 0.1), 0 3px 10px 0 rgba(0, 0, 0, 0.1);
  }
  .prompt-input {
    flex-basis: 49%;
    padding: 20px;
    background-color: rgba(44, 62, 80, 0.9);
    color: #ffffff;
    border-radius: 5px;
    box-shadow: 0 2px 4px 0 rgba(0, 0, 0, 0.1), 0 3px 10px 0 rgba(0, 0, 0, 0.1);
    width: 100%;
    border: none;
    font-family: 'Roboto', sans-serif;
    font-size: 16px;
    outline: none;
  }
  .prompt-input:focus {
    outline: none;
  }
  .prompt-input:hover {
    background-color: rgba(52, 73, 94, 0.9);
    cursor: pointer;
    transition: all 0.3s ease;
  }
  .lsf-richtext__line:hover {
    background: unset;
  }
  </Style>
  <Text name="chat" value="$text" layout="dialogue"/>
  <Header value="Question prompt:"/>
  <View className="prompt">
    <TextArea name="prompt" toName="chat" rows="4" editable="true" maxSubmissions="1" showSubmitButton="false"/>
  </View>
  <Header value="Proposed questions:"/>
  <TextArea name="response" toName="chat" rows="3" editable="true" maxSubmissions="1" showSubmitButton="false"/>
</View>
    '''
)

Load the dataset and insert it into Label Studio. The dataset is quite large, so we'll only load a few examples first. 

In [21]:
medal_dataset = load_dataset("medal", split='train')

In [30]:
# Insert 10 examples into Label Studio
num_examples = 10
for i in range(num_examples):
    task = medal_dataset[i]
    medal_questions_project.import_tasks(task)

For question generation, we need to have a strong prompt to yield solid results. Here is a useful prompt for generating medical questions for examples from the MeDAL dataset. 

```text
Given a block of medical text, generate several direct, succinct, and unique questions that stand alone, focusing on extracting specific medical information such as symptoms, diagnosis, treatment options, or patient management strategies. Each question should aim to elicit precise and informative responses without requiring additional context. The questions should cover diverse aspects of the medical content to ensure a comprehensive understanding. Ensure each question is clear and formulated to be self-contained. Here are examples to guide your question generation:

What are the common symptoms associated with [specific condition]?
How is [specific condition] diagnosed?
What treatment options are available for [specific condition]?
What are the potential side effects of [specific medication]?
What preventive measures are recommended for [specific condition]?

Use these examples as a template, tailoring questions to different parts of the text to maximize the dataset's utility and accuracy. Questions should be separated by a new line and not prefixed by any markers or numbers.
```

## Answer Generation with MeDAL
The final step involves setting up a project for answer generation using the questions created in the previous step.

We'll set up a project, export our questions generated in the previous section and generate answers in Label Studio.

In [109]:
medal_anwers_project = client.start_project(
    title='Project 3: MeDAL Answer Generation',
    label_config='''
<View className="root">
  <Style>
  .root {
    font-family: 'Roboto', sans-serif;
    line-height: 1.6;
    background-color: #f0f0f0;
  }
  .container {
    margin: 0 auto;
    padding: 20px;
    background-color: #ffffff;
    border-radius: 5px;
    box-shadow: 0 4px 8px 0 rgba(0, 0, 0, 0.1), 0 6px 20px 0 rgba(0, 0, 0, 0.1);
  }
  .prompt {
    padding: 20px;
    background-color: #0084ff;
    color: #ffffff;
    border-radius: 5px;
    margin-bottom: 20px;
    box-shadow: 0 2px 4px 0 rgba(0, 0, 0, 0.1), 0 3px 10px 0 rgba(0, 0, 0, 0.1);
  }
  .prompt-input {
    flex-basis: 49%;
    padding: 20px;
    background-color: rgba(44, 62, 80, 0.9);
    color: #ffffff;
    border-radius: 5px;
    box-shadow: 0 2px 4px 0 rgba(0, 0, 0, 0.1), 0 3px 10px 0 rgba(0, 0, 0, 0.1);
    width: 100%;
    border: none;
    font-family: 'Roboto', sans-serif;
    font-size: 16px;
    outline: none;
  }
  .prompt-input:focus {
    outline: none;
  }
  .prompt-input:hover {
    background-color: rgba(52, 73, 94, 0.9);
    cursor: pointer;
    transition: all 0.3s ease;
  }
  .lsf-richtext__line:hover {
    background: unset;
  }
  </Style>
  <Text name="chat" value="$text" layout="dialogue"/>
  <Header value="Answer prompt:"/>
  <View className="prompt">
    <TextArea name="prompt" toName="chat" rows="4" editable="true" maxSubmissions="1" showSubmitButton="false"/>
  </View>
  <Header value="Proposed answer:"/>
  <TextArea name="response" toName="chat" rows="3" editable="true" maxSubmissions="1" showSubmitButton="false"/>
</View>
    '''
)

Export questions from our previous project. 

In [39]:
# Download questions from Label Studio
questions_tasks = medal_questions_project.get_labeled_tasks()
len(questions_tasks)

In [92]:
questions_tasks[0]['annotations'][0]['result'][1]

{'value': {'text': ['What is the role of velvet antler peptide (svap) in preventing cardiac fibrosis from pressure overload?\nHow does svap interact with TGFÎ² receptors and disrupt the TGFÎ² pathway?\nWhat effects did treatment with svap have on cardiac dysfunction, fibroblast activation, and fibrosis in rats undergoing TAC surgery?\nWhat specific molecular changes were observed in the heart tissue following TAC surgery and how were they impacted by svap treatment?\nHow does svap inhibit reactive oxygen species, CTGF expression, and the phosphorylation of SMAD and ERK in cardiac fibroblasts?\nWhat similarities were observed between the effects of angiotensin II and TAC surgery, and how did svap counteract these effects?\nWhat mechanisms did svap utilize to block the TGFÎ² pathway in cardiac fibroblasts and prevent fibrosis?']},
 'id': '4d836b17-25a6-4e9d-8d18-f2313d75b65c',
 'from_name': 'response',
 'to_name': 'chat',
 'type': 'textarea',
 'origin': 'manual'}

Format as a Hugging Face dataset.

In [132]:
import re

# Extract questions
def extract_questions_data(questions_tasks):
    data = []
    for task in questions_tasks:
        for result in task['annotations'][0]['result']:
            if result['from_name'] == 'response':
                # Extract the abstract_id
                abstract_id = task['data']['abstract_id']
                
                # Extract the question text and split by newlines to handle multiple questions
                questions = result['value']['text'][0].split('\n')
                
                # Store each question with its corresponding abstract_id
                for question in questions:
                    # Check if the question is not empty and contains at least one alphanumeric character
                    if question.strip() and re.search('[a-zA-Z0-9]', question):
                        data.append({'abstract_id': abstract_id, 'text': question})
                break
    return data

extracted_questions_data = extract_questions_data(questions_tasks)

questions_dataset = Dataset.from_dict({'abstract_id': [item['abstract_id'] for item in extracted_questions_data], 
                             'text': [item['text'] for item in extracted_questions_data]})

# Format a 

Review our dataset and insert it into our answers project. 

In [133]:
questions_dataset

Dataset({
    features: ['abstract_id', 'text'],
    num_rows: 78
})

In [121]:
# Upload the dataset to our Answers Project
for question in questions_dataset: 
    medal_anwers_project.import_tasks(question)

Similar to the questions curation, we also need a strong prompt for generating the answers to these questions. Here is a sample prompt that can be used. 

```text
You are a medical expert tasked with providing the most accurate and succinct answers to specific questions based on detailed medical data. Focus on precision and directness in your responses, ensuring that each answer is factual, concise, and to the point. Avoid unnecessary elaboration and prioritize accuracy over sounding confident. Here are some guidelines for your responses:

- Provide clear, direct answers without filler or extraneous details.
- Base your responses solely on the information available in the medical text provided.
- Ensure that your answers are straightforward and easy to understand, yet medically accurate.
- Avoid speculative or generalized statements that are not directly supported by the text.

Use these guidelines to formulate your answers to the questions presented. 
```

## Curate Q&A Dataset
Once question-answer pairs are generated and refined, download the dataset. For a more robust use case, we would also perform any necessary post-processing here such as formatting and anonymization. 

Finally, upload the curated dataset to Hugging Face, ensuring it meets their standards for public datasets. 

In [122]:
# Download questions from Label Studio
answers_tasks = medal_anwers_project.get_labeled_tasks()
len(answers_tasks)

78

In [141]:
# Extract questions
def extract_answers_data(answers_tasks):
    data = []
    for task in answers_tasks:
        for result in task['annotations'][0]['result']:
            if result['from_name'] == 'response':
                # Extract the abstract_id
                abstract_id = task['data']['abstract_id']
                
                # Extract the question text and split by newlines to handle multiple questions
                answer = result['value']['text'][0]
                question = task['data']['text']
                
                # Store each question with its corresponding abstract_id
                data.append({'abstract_id': abstract_id, 'question': question, 'answer': answer})
    return data

extracted_answers_data = extract_answers_data(answers_tasks)

qa_dataset = Dataset.from_dict({'abstract_id': [item['abstract_id'] for item in extracted_answers_data], 
                             'question': [item['question'] for item in extracted_answers_data],
                             'answer': [item['answer'] for item in extracted_answers_data]})

In [151]:
qa_dataset[0]

{'abstract_id': 14145090,
 'question': 'What is the role of velvet antler peptide (svap) in preventing cardiac fibrosis from pressure overload?',
 'answer': 'The role of velvet antler peptide (SVAP) in preventing cardiac fibrosis from pressure overload is not explicitly mentioned in the provided text.'}

In [ ]:
qa_dataset.push_to_hub("<HF_Username>/med-qa")